In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os

# Tải mô hình CLIP một lần để tránh tải lặp lại
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def encode_image(image_path):
    try:
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features[0].numpy()
    except Exception as e:
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None  # Trả về None nếu lỗi, để lọc sau

# Đọc danh sách khung hình từ thư mục
frame_base_dir = "data/key_frame"
video_base_dir = "data/video"
frame_embeddings = []

# Duyệt qua các thư mục con trong key_frame
for key_frame_dir in os.listdir(frame_base_dir):
    frame_dir_path = os.path.join(frame_base_dir, key_frame_dir)
    if os.path.isdir(frame_dir_path):
        # Lấy video_path từ tên thư mục (giả định khớp với tiền tố)
        video_name = f"{key_frame_dir}.mp4"
        video_path = os.path.join(video_base_dir, video_name).replace("\\", "/")  # Chuẩn hóa đường dẫn
        if not os.path.exists(video_path):
            print(f"Video {video_path} không tồn tại, bỏ qua.")
            continue

        # Lấy tất cả file khung hình trong thư mục
        frame_paths = [os.path.join(frame_dir_path, f).replace("\\", "/") for f in os.listdir(frame_dir_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        for frame_path in frame_paths:
            embedding = encode_image(frame_path)
            if embedding is not None:  # Chỉ thêm nếu thành công
                frame_embeddings.append({
                    "video_path": video_path,
                    "frame_path": frame_path,
                    "vector": embedding
                })

print(f"Tổng số khung hình được xử lý: {len(frame_embeddings)}")

config.json: 0.00B [00:00, ?B/s]

d:\Big_project_2025\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\huggingface_cache\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Tổng số khung hình được xử lý: 327


# setup Lưu trữ Embeddings và Metadata trong PostgreSQL và Milvus

In [ ]:
import psycopg2
import os
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, MilvusClient
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Kết nối PostgreSQL
db_params = {
    "dbname": "video",
    "user": "postgres",
    "password": "123",
    "host": "localhost",
    "port": "5432"
}
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Tạo bảng với ràng buộc duy nhất
cur.execute("""
    CREATE TABLE IF NOT EXISTS videos (
        id SERIAL PRIMARY KEY,
        video_path VARCHAR(255) UNIQUE,
        title VARCHAR(255),
        description TEXT
    )
""")
cur.execute("""
    CREATE TABLE IF NOT EXISTS frame_mappings (
        frame_id SERIAL PRIMARY KEY,
        video_id INTEGER REFERENCES videos(id),
        frame_path VARCHAR(255),
        milvus_id BIGINT
    )
""")

In [3]:
from pymilvus import MilvusClient

# Kết nối tới Milvus server
milvus_client = MilvusClient(uri="http://localhost:19530")

# Kiểm tra kết nối bằng cách liệt kê collection
try:
    collections = milvus_client.list_collections()
    print("Kết nối thành công! Danh sách collection:", collections)
except Exception as e:
    print("Kết nối thất bại:", str(e))


Kết nối thành công! Danh sách collection: ['video_search', 'Movies', 'image_collection', 'my_rag_collection']


In [ ]:

collection_name = "video_search"
if client.has_collection(collection_name):
    client.drop_collection(collection_name)

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=512),
    FieldSchema(name="frame_path", dtype=DataType.VARCHAR, max_length=255)
]
schema = CollectionSchema(fields=fields, description="Embeddings khung hình video")
collection = Collection(name=collection_name, schema=schema)

index_params = {"metric_type": "IP", "index_type": "IVF_FLAT", "params": {"nlist": 1024}}
collection.create_index(field_name="vector", index_params=index_params)
collection.load()

# Chèn dữ liệu
frame_base_dir = "data/key_frame"
video_base_dir = "data/video"
for key_frame_dir in os.listdir(frame_base_dir):
    frame_dir_path = os.path.join(frame_base_dir, key_frame_dir)
    if os.path.isdir(frame_dir_path):
        video_name = f"{key_frame_dir}.mp4"
        video_path = os.path.join(video_base_dir, video_name).replace("\\", "/")
        if not os.path.exists(video_path):
            print(f"Video {video_path} không tồn tại, bỏ qua.")
            continue

        cur.execute("""
            INSERT INTO videos (video_path, title, description)
            VALUES (%s, %s, %s) ON CONFLICT (video_path) DO NOTHING RETURNING id
        """, (video_path, os.path.basename(video_path), "Video mẫu"))
        video_id = cur.fetchone()
        if video_id:
            video_id = video_id[0]
        else:
            cur.execute("SELECT id FROM videos WHERE video_path = %s", (video_path,))
            video_id = cur.fetchone()[0]

        frame_paths = [os.path.join(frame_dir_path, f).replace("\\", "/") for f in os.listdir(frame_dir_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        for frame_path in frame_paths:
            cur.execute("""
                INSERT INTO frame_mappings (video_id, frame_path)
                VALUES (%s, %s) ON CONFLICT (frame_path) DO NOTHING RETURNING frame_id
            """, (video_id, frame_path))
            frame_id = cur.fetchone()

            image = Image.open(frame_path)
            inputs = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")(images=image, return_tensors="pt")
            with torch.no_grad():
                embedding = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").get_image_features(**inputs)[0].numpy()

            data = {"vector": embedding, "frame_path": frame_path}
            mr = collection.insert([data])
            milvus_id = mr.primary_keys[0]
            cur.execute("""
                UPDATE frame_mappings
                SET milvus_id = %s
                WHERE frame_path = %s
            """, (milvus_id, frame_path))

conn.commit()
print("Dữ liệu đã được chèn và commit.")


d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google

Dữ liệu đã được chèn và commit.


# Tìm kiếm Image-to-Video

In [7]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import psycopg2
from pymilvus import connections, MilvusClient

# Kết nối PostgreSQL
db_params = {
    "dbname": "video",
    "user": "postgres",
    "password": "123",
    "host": "localhost",
    "port": "5432"
}
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Kết nối Milvus
try:
    connections.connect(host="localhost", port="19530")
    print("Kết nối Milvus thành công!")
except Exception as e:
    print(f"Lỗi kết nối Milvus: {e}")
    exit()

client = MilvusClient(uri="http://localhost:19530")

# Tải mô hình CLIP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Hàm tạo embedding từ hình ảnh
def encode_image(image_path):
    try:
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        return image_features[0].cpu().numpy()
    except Exception as e:
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None

# Hàm tìm kiếm video
def search_videos_by_image(image_path, top_k=10):
    query_embedding = encode_image(image_path)
    if query_embedding is None:
        print("Không thể tạo embedding cho hình ảnh.")
        return []
    search_results = client.search(
        collection_name="video_search",
        data=[query_embedding],
        limit=top_k,
        output_fields=["frame_path"]
    )
    return search_results



Kết nối Milvus thành công!
Sử dụng thiết bị: cpu


In [8]:
# Thực hiện tìm kiếm
image_path = "data/key_frame/L21_V001/015.jpg"  # Thay bằng đường dẫn ảnh bạn muốn query
results = search_videos_by_image(image_path)
for result in results[0]:
    frame_path = result["entity"]["frame_path"]
    cur.execute("""
        SELECT v.video_path, v.title
        FROM frame_mappings fm
        JOIN videos v ON fm.video_id = v.id
        WHERE fm.frame_path = %s
    """, (frame_path,))
    video_info = cur.fetchone()
    if video_info:
        print(f"Video: {video_info[1]}, Đường dẫn: {video_info[0]}, Độ tương đồng: {result['distance']}")
    else:
        print(f"Không tìm thấy video cho frame_path: {frame_path}")

# Đóng kết nối
conn.close()

Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 111.22276306152344
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 102.9091796875
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 85.33064270019531
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 83.57435607910156
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 81.2811050415039
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 80.79733276367188
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 80.4063949584961
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 80.2515640258789
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 79.47163391113281
Video: L21_V001.mp4, Đường dẫn: data/video/L21_V001.mp4, Độ tương đồng: 78.57772827148438


# text to video

In [39]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import torch.nn.functional as F
import psycopg2
from pymilvus import connections, MilvusClient

# Sử dụng context manager cho kết nối PostgreSQL
with psycopg2.connect(
    dbname="video",
    user="postgres",
    password="123",
    host="localhost",
    port="5432"
) as conn:
    with conn.cursor() as cur:
        # Kết nối Milvus
        try:
            connections.connect(host="localhost", port="19530")
            print("Kết nối Milvus thành công!")
        except Exception as e:
            print(f"Lỗi kết nối Milvus: {e}")
            exit()

        client = MilvusClient(uri="http://localhost:19530")

        # Tải mô hình CLIP
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Sử dụng thiết bị: {device}")
        model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        # Hàm tạo embedding từ text
        def encode_text(text):
            try:
                inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True, max_length=77).to(device)
                with torch.no_grad():
                    text_features = model.get_text_features(**inputs)
                # Chuẩn hóa embedding
                normalized_features = F.normalize(text_features, p=2, dim=1)[0].cpu().numpy()
                return normalized_features
            except Exception as e:
                print(f"Lỗi khi xử lý text: {e}")
                return None

        # Hàm tìm kiếm video dựa trên text
        def search_videos_by_text(text_query, top_k=10):
            text_embedding = encode_text(text_query)
            if text_embedding is None:
                print("Không thể tạo embedding cho text.")
                return []
            search_results = client.search(
                collection_name="video_search",
                data=[text_embedding],
                limit=top_k,
                output_fields=["frame_path"]
            )
            return search_results

        # Hàm lấy video từ frame_path
        def get_video_from_frame(frame_path):
            cur.execute("""
                SELECT v.video_path, v.title
                FROM frame_mappings fm
                JOIN videos v ON fm.video_id = v.id
                WHERE fm.frame_path = %s
            """, (frame_path,))
            video_info = cur.fetchone()
            return video_info

        # Thực hiện tìm kiếm dựa trên text
        text_query = "Pomelo tree"  # Thay bằng mô tả bạn muốn
        results = search_videos_by_text(text_query, top_k=5)  # Lấy 5 kết quả tốt nhất
        if results:
            for result in results[0]:
                frame_path = result["entity"]["frame_path"]
                video_info = get_video_from_frame(frame_path)
                if video_info:
                    print(f"Video: {video_info[1]}, Đường dẫn: {video_info[0]}, Độ tương đồng: {result['distance']}")
                else:
                    print(f"Không tìm thấy video cho frame_path: {frame_path}")
        else:
            print("Không tìm thấy kết quả nào.")

Kết nối Milvus thành công!
Sử dụng thiết bị: cpu
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 3.0837035179138184
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 3.019639015197754
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 2.9320945739746094
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 2.86421537399292
Video: L21_V002.mp4, Đường dẫn: data/video/L21_V002.mp4, Độ tương đồng: 2.8333077430725098


In [47]:
import pandas as pd
import os

# Load CSV mapping (có cột: n, pts_time, fps, frame_idx)
mapping_df = pd.read_csv(r"D:\Big_project_2025\Video_Similarity_Search\data\L21_V002.csv")

def get_time_from_keyframe(frame_path):
    """Lấy pts_time từ tên keyframe."""
    filename = os.path.splitext(os.path.basename(frame_path))[0]  # "067"
    try:
        n = int(filename)  # dùng cột n trong CSV
    except ValueError:
        return None

    row = mapping_df[mapping_df["n"] == n]
    if not row.empty:
        return float(row["pts_time"].values[0])
    else:
        return None

def group_timestamps(timestamps, gap_threshold=10.0):
    """Gom nhiều timestamp gần nhau thành khoảng [start, end]."""
    if not timestamps:
        return []
    timestamps = sorted(timestamps)
    ranges = []
    start = timestamps[0]
    end = timestamps[0]

    for t in timestamps[1:]:
        if t - end <= gap_threshold:
            end = t
        else:
            ranges.append((start, end))
            start = t
            end = t
    ranges.append((start, end))
    return ranges

def format_time(seconds):
    """Chuyển giây -> phút:giây (MM:SS)."""
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m}:{s:02d}"

# ================== Demo ==================
timestamps = []

print("📌 KẾT QUẢ CHI TIẾT TỪ MILVUS")
for result in results[0]:
    frame_path = result["entity"]["frame_path"]
    ts = get_time_from_keyframe(frame_path)
    if ts is not None:
        distance = result['distance']
        print(f"🎬 Frame: {frame_path} | ⏱ {ts:.2f}s ({format_time(ts)}) | 🔍 Độ tương đồng: {distance:.4f}")
        timestamps.append(ts)

# Gom các frame gần nhau thành khoảng
time_ranges = group_timestamps(timestamps, gap_threshold=15.0)

print("\n📌 KHOẢNG THỜI GIAN GOM LẠI")
for start, end in time_ranges:
    print(f"👉 Xuất hiện từ {format_time(start)} ({start:.2f}s) đến {format_time(end)} ({end:.2f}s)")


📌 KẾT QUẢ CHI TIẾT TỪ MILVUS
🎬 Frame: data/key_frame/L21_V002/077.jpg | ⏱ 279.13s (4:39) | 🔍 Độ tương đồng: 3.0837
🎬 Frame: data/key_frame/L21_V002/067.jpg | ⏱ 238.07s (3:58) | 🔍 Độ tương đồng: 3.0196
🎬 Frame: data/key_frame/L21_V002/070.jpg | ⏱ 254.33s (4:14) | 🔍 Độ tương đồng: 2.9321
🎬 Frame: data/key_frame/L21_V002/079.jpg | ⏱ 283.57s (4:43) | 🔍 Độ tương đồng: 2.8642
🎬 Frame: data/key_frame/L21_V002/068.jpg | ⏱ 242.57s (4:02) | 🔍 Độ tương đồng: 2.8333

📌 KHOẢNG THỜI GIAN GOM LẠI
👉 Xuất hiện từ 3:58 (238.07s) đến 4:14 (254.33s)
👉 Xuất hiện từ 4:39 (279.13s) đến 4:43 (283.57s)
